<a href="https://colab.research.google.com/github/bstrain71/422_MNIST/blob/master/Strain_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ingest

In [0]:
import pandas as pd
import math
from datetime import datetime
import csv

In [6]:
from google.colab import drive
#drive.mount('/content/drive')
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
test_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/MNIST/test.csv')
train_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/MNIST/train.csv')

In [8]:
print(train_df.shape) # 42000x785? OK
print(test_df.shape) # 28000x784? OK

(42000, 785)
(28000, 784)


## (1) Begin by fitting a random forest classifier using the full set of 784 explanatory variables and the model training set (train.csv). Record the time it takes to fit the model and then evaluate the model on the test.csv data by submitting to Kaggle.com. Provide your Kaggle.com score and user ID.


In [0]:
X_train = train_df.drop(columns=['label'])
y_train = train_df['label']

(1) Begin by fitting a random forest classifier using the full set of 784 explanatory variables and the model training set (train.csv). Record the time it takes to fit the model and then evaluate the model on the test.csv data by submitting to Kaggle.com. Provide your Kaggle.com score and user ID.

In [10]:
# calculate recommended max depth and max features
math.log2(784) # max features is the square root of number of predictors


9.614709844115207

In [11]:
from sklearn.ensemble import RandomForestClassifier

start=datetime.now()
rforest = RandomForestClassifier(n_estimators = 100,
                                 max_features = 10).fit(X_train, y_train)
end=datetime.now()
print(end-start)

0:00:13.374553


In [0]:
rforest_predictions = pd.DataFrame(rforest.predict(test_df))
rforest_predictions['ImageId'] = test_df.index + 1
rforest_predictions = rforest_predictions.rename(columns={0:"Label"})
rforest_predictions = rforest_predictions.reindex(columns=['ImageId','Label'])



In [36]:
print(rforest_predictions.columns)

Index(['Label', 'ImageId'], dtype='object')


In [41]:
rforest_predictions.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [0]:
export_csv = rforest_predictions.to_csv (r'/content/randomforest_results.csv',
                        index = None,
                        header=True)

Kaggle.com Info

Score: 0.96271

Username: brandonian (https://www.kaggle.com/brandonian)

Model train time: 0:00:15.281019

## (2) Execute principal components analysis (PCA) on the combined training and test set data together, generating principal components that represent 95 percent of the variability in the explanatory variables. The number of principal components in the solution should be substantially fewer than the 784 explanatory variables. Record the time it takes to identify the principal components.

In [33]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Combine test and training sets ** this is wrong **
extra_col = test_df
extra_col['label'] = 0

combined_df = train_df.append(extra_col['label'] + test_df)

In [0]:
combined_df.shape

In [0]:
from yellowbrick.features.pca import PCADecomposition

visualizer = PCADecomposition(scale=True, color=colors)
visualizer.fit_transform(X_train, y)
visualizer.poof()